In [ ]:
import json

prompts, names, runs = {}, {}, []

with open("main.json.log", "r") as file:
    for line in file:
        item = json.loads(line)

        if item["type"] == "name":
            names[item["key"]] = item["name"]
        elif item["type"] == "prompt" and item.get("main"):
            prompts[item["prompt_id"]] = item["prompt"]
        elif item["type"] == "run" and "result" in item:
            runs.append(item)

In [3]:
import pandas as pd
import json

def get_text(result):
    try:
        return result["result"]["message"]["content"]
    except Exception:
        try:
            return result["result"]["choices"][0]["message"]["content"]
        except Exception:
            return None

with open("out-results.json.log", "r") as file:
    results = []

    for line in file:
        result = json.loads(line)
        model = result.get("model")
        runner = "oai" if (model in oai_models) else result.get("runner")
        prompt_id = result.get("task_id")

        if "result" in result and prompt_id in prompts_corpus and "elapsed_time" in result and runner in runners and model in models:
            results.append({
                "runner": runners[runner],
                "model": models[model],
                "_runner": runner,
                "_model": model,
                "prompt_id": prompt_id,
                "prompt": prompts[prompt_id],
                "answer": get_text(result),
                "total_len": len(prompts[prompt_id]) + len(get_text(result)),
                "elapsed_time": result["elapsed_time"]
            })

df = pd.DataFrame(results)

In [ ]:
df_count = df.groupby(["runner", "model"]).size().reset_index(name="num_items")
df_count.pivot(index="model", columns="runner", values="num_items")

In [ ]:
df_filtered = df.groupby(["runner", "model"]).filter(lambda g: len(g) == len(prompts_corpus))
df_filtered["ratio"] = df_filtered["total_len"] / df_filtered["elapsed_time"]

df_median = df_filtered.groupby(["runner", "model"])["ratio"].median().reset_index()
table = df_median.pivot(index="model", columns="runner", values="ratio")

table.index.name = None
table.columns.name = None

runners_sorted = [r for r in runners.values() if r in table.columns]
models_sorted = [r for r in models.values() if r in table.index]

table = table.loc[models_sorted, runners_sorted]
table.applymap(lambda x: int(x) if pd.notna(x) else "")